In [1]:
import pandas as pd 
import os
from matplotlib import pyplot as plt

In [2]:
BASE_PATH = os.path.join(os.getcwd(), os.pardir)
DATA_PATH = os.path.join(BASE_PATH, 'data')

In [3]:
train_df = pd.read_json(os.path.join(DATA_PATH, 'train.json'))

# Extract text/numerical/categorical features

In [ ]:
features = ['bathrooms', 'bedrooms', 'building_id', 'month', 'day', 'hour', 'weekday', 'latitude', 'longitude', 
            'manager_id', 'num_photos', 'price', 'address']

In [15]:
one_hot_features = set()

In [16]:
for listing in train_df['features'].values:
    for feature in listing:
        one_hot_features.add(feature)

In [17]:
one_hot_features

{'2 year lease',
 '** SPACIOUS STUDIO SUPREME * AMAZING CORNER LAYOUT * TONS OF SUNLIGHT * PETS OK **',
 'deck',
 '24-hour doorman',
 'Yoga an Pilates class',
 'Large finished basement',
 '000-square foot sun deck• Free WiFi in Marc Club',
 'On-site lifestyle concierge by Luxury Attaché',
 'Quiet Neighborhood',
 'Yoga and Spin Studios',
 'Washer in Unit',
 'Outdoor Basketball Court',
 'Sparkling  Marble Bathroom',
 '1 Month Free',
 'onsite valet',
 'Shared backyard',
 'Rooftop deck',
 'backyard',
 'Hot Water Inc. HUGE Walk-In Closet or Extra Room!',
 'Natural Sunlight',
 'walk-up',
 'Garage Parking',
 'Furnished',
 'heat and hot water included',
 'featuring a sophisticated Precor® online profile system that tracks your personal fitness goals',
 'Convinient to all major transportation',
 'Indoor Pool',
 'modern kitchen',
 'FREE MONTH & NO-FEE',
 'ACT FAST WILL NOT LAST',
 'Pre War',
 'Hardwood floors',
 '** HUGE TRUE 3BR HOME * MR CLEAN APPROVED * MASSIVE WINDOWS GALORE * STEPS TO THE P